## Check Buildstock sampling per 15 SF prototypes (Elevate Energy)

In [1]:
import pandas as pd
import numpy as np
import os
print(f'Notebook path: {os.getcwd()}')

Notebook path: /Users/lliu2/Documents/GitHub/OpenStudio-BuildStock/files


### Download results from S3:
://S3/resbldg-datasets/chicagoeui

In [2]:
# local path to downloaded results
iteration = '_11' # <----- options: 'base', '_01'~'_11', '_national_2018'
result_path = '/Users/lliu2/Documents/Chicago retrofits/ResStock results/'
iteration_path = f'cookcnty_2007{iteration}' 
baseline_result = os.path.join(result_path,iteration_path,'buildstock.csv')
print(f'Result and file storage path: {iteration_path}')


Result and file storage path: cookcnty_2007_11


In [3]:
res = pd.read_csv(baseline_result)
res

,Building,Location,Usage Level,Corridor,Vacancy Status,Eaves,Door Area,Overhangs,Interior Shading,Doors,...,Misc Freezer,Misc Hot Tub Spa,Misc Pool,Misc Pool Heater,Misc Pool Pump,HVAC Has Ducts,Ducts,Infiltration,Geometry Perimeter Footprint Ratio,Chicago
0,1,IL_Chicago-OHare.Intl.AP.725300,Medium,Double-Loaded Interior,Occupied,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,...,None,None,None,None,None,Yes,"20% Leakage, Uninsulated",15 ACH50,0.13,Non-City Cook County
1,2,IL_Chicago-OHare.Intl.AP.725300,High,Double-Loaded Interior,Occupied,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,...,None,None,None,None,None,Yes,"20% Leakage, Uninsulated",40 ACH50,0.13,City of Chicago
2,3,IL_Chicago-OHare.Intl.AP.725300,Low,Double-Loaded Interior,Occupied,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,...,"EF 12, National Average",None,None,None,None,Yes,In Finished Space,8 ACH50,0.13,Non-City Cook County
3,4,IL_Chicago-OHare.Intl.AP.725300,Low,Double-Loaded Interior,Occupied,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,...,None,None,None,None,None,Yes,"20% Leakage, R-4",15 ACH50,0.13,Non-City Cook County
4,5,IL_Chicago-OHare.Intl.AP.725300,Low,Double-Loaded Interior,Occupied,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,...,None,None,None,None,None,Yes,"20% Leakage, R-4",15 ACH50,0.13,Non-City Cook County
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,39996,IL_Chicago-OHare.Intl.AP.725300,Low,Double-Loaded Interior,Occupied,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,...,None,None,None,None,None,Yes,"20% Leakage, Uninsulated",25 ACH50,0.13,Non-City Cook County
39996,39997,IL_Chicago-OHare.Intl.AP.725300,Medium,Double-Loaded Interior,Occupied,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,...,None,None,None,None,None,Yes,"20% Leakage, R-4",25 ACH50,0.13,Non-City Cook County
39997,39998,IL_Chicago-OHare.Intl.AP.725300,Low,Double-Loaded Interior,Occupied,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,...,None,None,None,None,None,Yes,"20% Leakage, R-4",25 ACH50,0.13,City of Chicago
39998,39999,IL_Chicago-OHare.Intl.AP.725300,Low,Double-Loaded Interior,Occupied,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,...,"EF 12, National Average",None,None,None,None,Yes,"30% Leakage, Uninsulated",5 ACH50,0.13,City of Chicago


In [4]:
if iteration == '_national_2018':
    res = res[res['AHS Region']=='CBSA Chicago-Naperville-Elgin, IL-IN-WI'].reset_index(drop=True)
    res

In [5]:
### HVAC cols changed after HVAC restructuring (implemented in Run5)
if iteration in ['base','_01','_02','_03','_04']:
    cooling_col = 'HVAC System Cooling Type'
else:
    cooling_col = 'HVAC Cooling Type'
if iteration in ['base','_01','_02','_03','_04']:
    heating_col = 'HVAC System Heating Natural Gas'
else:
    heating_col = 'HVAC Heating Type And Fuel'
print('selected')


selected


### Check housing charateristics distributions

In [29]:
proto_chars = ['Geometry Stories',
              'Geometry Wall Type',
              'Vintage ACS',
              cooling_col,
              heating_col,
              'Geometry Floor Area'
             ]

Ns = len(res);
print(f'>>> ResStock - {iteration_path} - RESULT SUMMARY:\n')
print(f'  * {Ns} samples ran created \n')

print('>>> Housing characteristics splits:\n')

Res_char = []
for i, char in enumerate(proto_chars,1):
    Nchar = res.groupby(char)['Building'].count()
    Nchar = Nchar/Ns
    print(f'  * [{i}] {Nchar}\n')
    
    ## append for export
    Nchar = Nchar.rename('fraction').to_frame()
    Nchar['housing_char'] = Nchar.index.name
    Res_char.append(Nchar)
    
Res_char = pd.concat(Res_char, axis=0)
Res_char.index.name = 'sub_char'
Res_char = Res_char.reset_index()
Res_char = Res_char[['housing_char','sub_char','fraction']]

Res_char.to_csv(os.path.join(result_path, iteration_path, 'buildstock_housing_char_breakdown.csv'), index=False)
print(f'>>> file saved to:\n  {os.path.join(result_path, iteration_path)}')

>>> ResStock - cookcnty_2007_11 - RESULT SUMMARY:

  * 40000 samples ran created 

>>> Housing characteristics splits:

  * [1] Geometry Stories
1    0.506775
2    0.282675
3    0.210550
Name: Building, dtype: float64

  * [2] Geometry Wall Type
Masonry     0.71495
WoodStud    0.28505
Name: Building, dtype: float64

  * [3] Vintage ACS
1940-59    0.235025
1960-79    0.255075
1980-99    0.137875
2000-09    0.076125
2010s      0.011675
<1940      0.284225
Name: Building, dtype: float64

  * [4] HVAC Cooling Type
Central AC    0.61605
Heat Pump     0.02560
None          0.05250
Room AC       0.30585
Name: Building, dtype: float64

  * [5] HVAC Heating Type And Fuel
Electricity ASHP                       0.020250
Electricity Baseboard                  0.044775
Electricity Electric Boiler            0.000750
Electricity Electric Furnace           0.028950
Electricity Electric Wall Furnace      0.000900
Electricity Shared Heating             0.035150
Fuel Oil Fuel Boiler                   0.

### Add additional metrics to ResStock results

In [14]:
# (1) add sqft and eui
sqft_map = {'0-499': 328,
            '500-749': 633,
            '750-999': 885,
            '1000-1499': 1220,
            '1500-1999': 1690,
            '2000-2499': 2176,
            '2500-2999': 2663,
            '3000-3999': 3301,
            '4000+': 8194}

res['sqft'] = res['Geometry Floor Area'].map(sqft_map)

## EE prototype tags
res['vintage_ee'] = 'post-1978'
res.loc[res['Vintage ACS'].isin(['1940-59','1960-79']),'vintage_ee'] = '1942-1978'
res.loc[res['Vintage ACS']=='<1940','vintage_ee'] = 'pre-1942'

res['stories'] = '2 or more stories'
res.loc[res['Geometry Stories']==1,'stories'] = 'less than 2 stories'

print('new cols added to "res"')

new cols added to "res"


In [12]:
### get sqft distr by building type and vintage
res_sqft_distr = res.groupby(['Geometry Building Type RECS', 'Vintage ACS',
    'Geometry Floor Area'])['Building'].count().to_frame()

# normalize
res_sqft_distr = res_sqft_distr.groupby(level = [0,1])['Building'].apply(lambda x: x/x.sum()).unstack(level=2)

# export
#res_sqft_distr.to_csv(os.path.join(result_path, iteration_path, 'building_type_vintage_floor_area.csv'), index=True)
#print('sqft distr by vintage & bldg type exported')
res_sqft_distr

Geometry Floor Area                           0-499  1000-1499  1500-1999  \
Geometry Building Type RECS   Vintage ACS                                   
Mobile Home                   1940-59           NaN   1.000000        NaN   
                              1960-79           NaN   0.494505   0.175824   
                              1980-99           NaN   0.401961   0.196078   
                              2000-09           NaN   0.259259   0.259259   
                              2010s        0.125000   0.125000   0.125000   
                              <1940        0.105263   0.105263   0.105263   
Multi-Family with 2 - 4 Units 1940-59      0.025373   0.294776   0.224627   
                              1960-79      0.095652   0.334783        NaN   
                              1980-99      0.052270   0.374140   0.094911   
                              2000-09           NaN   0.451253   0.211699   
                              2010s             NaN   0.423077        NaN   
                              <1940        0.059256   0.473555   0.141283   
Multi-Family with 5+ Units    1940-59      0.080925   0.260116   0.081503   
                              1960-79      0.046192   0.266167   0.031211   
                              1980-99      0.101802   0.204054   0.062613   
                              2000-09      0.045662   0.264188   0.153947   
                              2010s        0.217712   0.158672        NaN   
                              <1940        0.215633   0.205596   0.056873   
Single-Family Attached        1940-59           NaN   1.000000        NaN   
                              1960-79           NaN   0.426877   0.239130   
                              1980-99           NaN   0.416883   0.264935   
                              2000-09           NaN   0.336182   0.336182   
                              2010s             NaN   0.250000   0.428571   
                              <1940        0.343434   0.303030   0.257576   
Single-Family Detached        1940-59      0.005005   0.326493   0.269436   
                              1960-79      0.002921   0.246012   0.326219   
                              1980-99           NaN   0.130307   0.228774   
                              2000-09      0.021390   0.054813   0.145722   
                              2010s             NaN   0.120370   0.120370   
                              <1940             NaN   0.254762   0.211376   

Geometry Floor Area                        2000-2499  2500-2999  3000-3999  \
Geometry Building Type RECS   Vintage ACS                                    
Mobile Home                   1940-59            NaN        NaN        NaN   
                              1960-79            NaN        NaN        NaN   
                              1980-99            NaN        NaN        NaN   
                              2000-09            NaN        NaN        NaN   
                              2010s         0.125000   0.125000   0.125000   
                              <1940         0.157895   0.105263   0.105263   
Multi-Family with 2 - 4 Units 1940-59       0.023134        NaN        NaN   
                              1960-79            NaN        NaN        NaN   
                              1980-99            NaN        NaN        NaN   
                              2000-09       0.089136        NaN        NaN   
                              2010s         0.192308        NaN        NaN   
                              <1940         0.033301   0.012733   0.026200   
Multi-Family with 5+ Units    1940-59            NaN        NaN   0.024277   
                              1960-79            NaN   0.008240        NaN   
                              1980-99       0.037838        NaN        NaN   
                              2000-09       0.133725        NaN        NaN   
                              2010s              NaN        NaN        NaN   
                              <1940         0.072689 

In [97]:
### get foundation type distr by building type, vintage

res_found_distr = res[res['build_existing_model.geometry_building_type_recs'] =="Single-Family Detached"].groupby(
    ['build_existing_model.vintage_acs', 'build_existing_model.geometry_stories',
    'build_existing_model.geometry_foundation_type'])['Building'].count().to_frame()

# normalize
res_found_distr = res_found_distr.groupby(level = [0,1])['Building'].apply(lambda x: x/x.sum()).unstack(level=2)
res_found_distr



build_existing_model.geometry_foundation_type                              Crawl  \
build_existing_model.vintage_acs build_existing_model.geometry_stories             
1940-59                          1                                      0.126927   
                                 2                                      0.134772   
                                 3                                      0.147059   
1960-79                          1                                      0.158642   
                                 2                                      0.164084   
                                 3                                      0.164179   
1980-99                          1                                      0.192175   
                                 2                                      0.209333   
                                 3                                      0.206897   
2000-09                          1                                      0.117834   
                                 2                                      0.088517   
                                 3                                      0.100000   
2010s                            1                                      0.071429   
                                 2                                      0.109375   
                                 3                                      0.500000   
<1940                            1                                      0.161243   
                                 2                                      0.167607   
                                 3                                      0.153153   

build_existing_model.geometry_foundation_type                           Heated Basement  \
build_existing_model.vintage_acs build_existing_model.geometry_stories                    
1940-59                          1                                             0.428950   
                                 2                                             0.401400   
                                 3                                             0.382353   
1960-79                          1                                             0.353704   
                                 2                                             0.390155   
                                 3                                             0.328358   
1980-99                          1                                             0.321059   
                                 2                                             0.322667   
                                 3                                             0.344828   
2000-09                          1                                             0.528662   
                                 2                                             0.586124   
                                 3                                             0.400000   
2010s                            1                                             0.595238   
                                 2                                             0.515625   
                                 3                                             0.500000   
<1940                            1                                             0.284024   
                                 2                                             0.277898   
                                 3                                             0.234234   

build_existing_model.geometry_foundation_type                           Pier and Beam  \
build_existing_model.vintage_acs build_existing_model.geometry_stories                  
1940-59                          1                                           0.014933   
                                 2                                           0.015169   
                                 3                                           0.029412   
1960-79                          1                   

In [98]:
### get foundation type distr by building type, vintage (EE prototype)

res_found_distr = res[res['build_existing_model.geometry_building_type_recs'] =="Single-Family Detached"].groupby(
    ['vintage_ee', 'stories',
    'build_existing_model.geometry_foundation_type'])['Building'].count().to_frame()

# normalize
res_found_distr = res_found_distr.groupby(level = [0,1])['Building'].apply(lambda x: x/x.sum()).unstack(level=2)
res_found_distr


build_existing_model.geometry_foundation_type     Crawl  Heated Basement  \
vintage_ee stories                                                         
1942-1978  2 or more stories                   0.146640         0.395112   
           less than 2 stories                 0.140828         0.395969   
post-1978  2 or more stories                   0.164179         0.420267   
           less than 2 stories                 0.168980         0.383673   
pre-1942   2 or more stories                   0.166943         0.275891   
           less than 2 stories                 0.161243         0.284024   

build_existing_model.geometry_foundation_type  Pier and Beam      Slab  \
vintage_ee stories                                                       
1942-1978  2 or more stories                        0.011541  0.209776   
           less than 2 stories                      0.011499  0.211310   
post-1978  2 or more stories                        0.003142  0.238806   
           less than 2 stories                      0.004898  0.267755   
pre-1942   2 or more stories                        0.010771  0.078708   
           less than 2 stories                      0.008876  0.063609   

build_existing_model.geometry_foundation_type  Unheated Basement  
vintage_ee stories                                                
1942-1978  2 or more stories                            0.236931  
           less than 2 stories                          0.240395  
post-1978  2 or more stories                            0.173606  
           less than 2 stories                          0.174694  
pre-1942   2 or more stories                            0.467688  
           less than 2 stories                          0.482249

In [168]:
res[res['build_existing_model.geometry_building_type_recs'] == "Single-Family Detached"].groupby([
    'build_existing_model.geometry_floor_area_bin','build_existing_model.geometry_wall_type',
'build_existing_model.vintage_acs'])['Building'].count()

build_existing_model.geometry_floor_area_bin  build_existing_model.geometry_wall_type  build_existing_model.vintage_acs
0-1499                                        Masonry                                  1940-59                             1748
                                                                                       1960-79                              674
                                                                                       1980-99                               99
                                                                                       2000-09                               31
                                                                                       2010s                                  9
                                                                                       <1940                                681
                                              WoodStud                                 1940-59                  

In [16]:
res[res['Geometry Building Type RECS'] == "Single-Family Detached"].groupby([
    'stories','Geometry Wall Type',
'vintage_ee'])['sqft'].mean().unstack(level=-1)

vintage_ee                                1942-1978    post-1978     pre-1942
stories             Geometry Wall Type                                       
2 or more stories   Masonry             2345.281553  4057.108317  2549.022752
                    WoodStud            2281.485930  3614.343501  2322.710784
less than 2 stories Masonry             1629.042789  2386.508029  1460.060606
                    WoodStud            1616.292899  2243.300336  1402.300142

In [178]:
res[res['Geometry Building Type RECS'] == "Single-Family Detached"].groupby([
    'Geometry Wall Type','stories',
'Vintage'])['sqft'].mean().unstack(level=-1)

build_existing_model.vintage                                    1940s  \
build_existing_model.geometry_wall_type stories                         
Masonry                                 2 or more stories    0.771525   
                                        less than 2 stories  0.684101   
WoodStud                                2 or more stories    0.852331   
                                        less than 2 stories  0.761761   

build_existing_model.vintage                                    1950s  \
build_existing_model.geometry_wall_type stories                         
Masonry                                 2 or more stories    0.807702   
                                        less than 2 stories  0.690659   
WoodStud                                2 or more stories    0.898863   
                                        less than 2 stories  0.778996   

build_existing_model.vintage                                    1960s  \
build_existing_model.geometry_wall_type stories                         
Masonry                                 2 or more stories    0.716290   
                                        less than 2 stories  0.622227   
WoodStud                                2 or more stories    0.811908   
                                        less than 2 stories  0.671872   

build_existing_model.vintage                                    1970s  \
build_existing_model.geometry_wall_type stories                         
Masonry                                 2 or more stories    0.615688   
                                        less than 2 stories  0.557976   
WoodStud                                2 or more stories    0.621496   
                                        less than 2 stories  0.558558   

build_existing_model.vintage                                    1980s  \
build_existing_model.geometry_wall_type stories                         
Masonry                                 2 or more stories    0.437891   
                                        less than 2 stories  0.456554   
WoodStud                                2 or more stories    0.468157   
                                        less than 2 stories  0.426862   

build_existing_model.vintage                                    1990s  \
build_existing_model.geometry_wall_type stories                         
Masonry                                 2 or more stories    0.372117   
                                        less than 2 stories  0.371588   
WoodStud                                2 or more stories    0.385075   
                                        less than 2 stories  0.378791   

build_existing_model.vintage                                    2000s  \
build_existing_model.geometry_wall_type stories                         
Masonry                                 2 or more stories    0.298608   
                                        less than 2 stories  0.295283   
WoodStud                                2 or more stories    0.310635   
                                        less than 2 stories  0.316522   

build_existing_model.vintage                                    2010s  \
build_existing_model.geometry_wall_type stories                         
Masonry                                 2 or more stories    0.294071   
                                        less than 2 stories  0.305492   
WoodStud                                2 or more stories    0.285653   
                                        less than 2 stories  0.271593   

build_existing_model.vintage                                    <1940  
build_existing_model.geometry_wall_type stories                        
Masonry                                 2 or more stories    0.744136  
                                        less than 2 stories  0.748823  
WoodStud                                2 or more stories    0.863646  
                                        less than 2 stories  0.849221

In [18]:
res_by_foundation = res[(res['Geometry Building Type RECS'] == "Single-Family Detached")].groupby([
    'vintage_ee', 'stories', 'Geometry Wall Type',
    'Geometry Foundation Type'
])

res_by_foundation3 = res_by_foundation['Building'].count().unstack(level=0)
res_by_foundation4 = res_by_foundation['sqft'].mean().unstack(level=0)

res_by_foundation3['type'] = 'count'
res_by_foundation4['type'] = 'sqft'

res_by_foundation_all = pd.concat([res_by_foundation3, res_by_foundation4], axis=0)
#res_by_foundation_all.to_csv(os.path.join(result_path, iteration_path, 'building_type_vintage_foundation.csv'), index=True)
print('foundation type count, sqft by vintage & bldg type exported')

res_by_foundation_all.head(10*4)

foundation type eui, sqft, count distr by vintage & bldg type exported


vintage_ee                                                         1942-1978  \
stories             Geometry Wall Type Geometry Foundation Type                
2 or more stories   Masonry            Crawl                      147.000000   
                                       Heated Basement            556.000000   
                                       Pier and Beam               19.000000   
                                       Slab                       237.000000   
                                       Unheated Basement          277.000000   
                    WoodStud           Crawl                      227.000000   
                                       Heated Basement            477.000000   
                                       Pier and Beam               47.000000   
                                       Slab                       317.000000   
                                       Unheated Basement          389.000000   
less than 2 stories Masonry            Crawl                      686.000000   
                                       Heated Basement           2211.000000   
                                       Pier and Beam               83.000000   
                                       Slab                      1042.000000   
                                       Unheated Basement         1026.000000   
                    WoodStud           Crawl                      400.000000   
                                       Heated Basement            897.000000   
                                       Pier and Beam               66.000000   
                                       Slab                       587.000000   
                                       Unheated Basement          754.000000   
2 or more stories   Masonry            Crawl                     2305.816327   
                                       Heated Basement           2364.206835   
                                       Pier and Beam             1964.421053   
                                       Slab                      2491.042194   
                                       Unheated Basement         2229.649819   
                    WoodStud           Crawl                     2155.462555   
                                       Heated Basement           2266.241090   
                                       Pier and Beam             2426.659574   
                                       Slab                      2288.921136   
                                       Unheated Basement         2350.120823   
less than 2 stories Masonry            Crawl                     1662.183673   
                                       Heated Basement           1603.028946   
                                       Pier and Beam             1491.349398   
                                       Slab                      1657.849328   
                                       Unheated Basement         1644.826511   
                    WoodStud           Crawl                     1600.247500   
                                       Heated Basement           1620.656633   
                                       Pier and Beam             1615.909091   
                                       Slab                      1620.100511   
                                       Unheated Basement         1616.683024   

vintage_ee                                                         post-1978  \
stories             Geometry Wall Type Geometry Foundation Type                
2 or more stories   Masonry            Crawl                       66.000000   
                                       Heated Basement            174.000000   
                                       Pier and Beam               15.000000   
                                       Slab                       169.000000   
                                       Unheated Basement           93.000000   
                    WoodStud           Crawl                      103.000000   

In [165]:
vintage = ['2010s'] # <--- need to remove/change lines with arrows
res2 = res[res['build_existing_model.vintage'].isin(vintage)] # <--- need to remove/change lines with arrows

### Get results by building prototypes from Elevate Energy
#### Note: cannot add prototype tags to ResStock results as prototypes overlap in chars

In [19]:
## Elevate Energy prototype chars
groups = pd.read_csv(os.path.join(result_path,'Groups.csv'))
print('prototypes loaded')


prototypes loaded


In [24]:
def get_res_by_prototype(res, row):
    res_group_i = res[res['Geometry Stories'].isin(row['Stories'].split(',')) & \
        res['Geometry Wall Type'].isin(row['WallType'].split(',')) & \
        res['Vintage ACS'].isin(row['Vintage'].split(',')) & \
        res['Geometry Building Type RECS'].isin(['Single-Family Detached'])]
    
    return res_group_i
    
    
### (3) get count, median & mean sqft
Metric_map1 = {'count': '',
               'median sqft': 'sqft'}
Metric_map2 = {'mean sqft': 'sqft'}

## initialize
for metric in {**Metric_map1, **Metric_map2}:
    groups[metric] = np.NAN


for i, row in groups.iterrows():
    res_group_i = get_res_by_prototype(res, row)
    
    ### (3.1) get median values
    for metric, res_var in Metric_map1.items():
        if metric == 'count':
            groups.iloc[i, groups.columns.get_loc('count')] = \
                len(res_group_i)
        else:
            groups.iloc[i, groups.columns.get_loc(metric)] = \
                res_group_i[res_var].median()

    ### (3.2) get mean values
    for metric, res_var in Metric_map2.items():
        groups.iloc[i, groups.columns.get_loc(metric)] = \
            res_group_i[res_var].mean()

print('>> median & mean sqft & eui computed:');

groups.set_index(['HousingGroupNo','HousingGroupName'])[{**Metric_map1, **Metric_map2}]


>> median & mean sqft & eui computed:


,,count,median sqft,mean sqft
HousingGroupNo,HousingGroupName,,,
1,Masonry Post-1978 less than 2 stories,685.0,2176.0,2386.508029
2,Masonry All Years Split Level,9135.0,1690.0,1999.694691
3,Masonry Post-1978 2 or more stories,517.0,3301.0,4057.108317
4,Masonry 1942-1978 Less than 2 Stories,5048.0,1690.0,1629.042789
5,Masonry 1942-1978 2 or more stories,1236.0,2176.0,2345.281553
6,Masonry Pre-1942 less than 2 stories,726.0,1220.0,1460.060606
7,Masonry Pre-1942 2 or more stories,923.0,2176.0,2549.022752
8,Frame Post-1978 Less than 2 stories,596.0,2176.0,2243.300336
9,Frame Post-1978 Split Level,1350.0,2176.0,3009.053333


In [26]:
### (5) get heating & cooling and floor area char breakdowns

House = []
House_chars = [cooling_col, heating_col,
              'Infiltration',
              'Insulation Wall',
              'Insulation Unfinished Attic']

for i, row in groups.iterrows():
    
    res_group_i = get_res_by_prototype(res, row)
    
    Ns = len(res_group_i)
    Hi = []
    for char in House_chars:
        Nchar = res_group_i.groupby(char)['Building'].count()/Ns # fractions
        Nchar = Nchar.rename('fraction').to_frame()
        Nchar['housing_char'] = Nchar.index.name
        Hi.append(Nchar)
    
    Hi = pd.concat(Hi, axis=0)
    Hi.index.name = 'sub_char'
    Hi = Hi.reset_index()
    Hi['HousingGroupNo'] = row['HousingGroupNo']
    
    House.append(Hi)
    
House = pd.concat(House, axis=0)
House = pd.pivot_table(House, values='fraction', index=['HousingGroupNo'],
                    columns=['housing_char','sub_char'], aggfunc=np.sum)

print('computed')

computed


In [27]:
## Combine "groups" + "House" into "Out" for export
out = groups.drop(['Stories','WallType','Vintage'], axis=1)
out = pd.concat([out.set_index('HousingGroupNo')], axis=1, keys=['none']).join(House)
out

none  \
                                     HousingGroupName   
HousingGroupNo                                          
1               Masonry Post-1978 less than 2 stories   
2                       Masonry All Years Split Level   
3                 Masonry Post-1978 2 or more stories   
4               Masonry 1942-1978 Less than 2 Stories   
5                 Masonry 1942-1978 2 or more stories   
6                Masonry Pre-1942 less than 2 stories   
7                  Masonry Pre-1942 2 or more stories   
8                 Frame Post-1978 Less than 2 stories   
9                         Frame Post-1978 Split Level   
10                  Frame Post-1978 2 or more stories   
11                Frame 1942-1978 less than 2 stories   
12                         Frame Pre-1942 Split Level   
13                  Frame 1942-1978 2 or more stories   
14                 Frame Pre-1942 less than 2 stories   
15                   Frame Pre-1942 2 or more stories   

                                                                            \
               Non-normalized gas usage Non-normalized elec. usage   count   
HousingGroupNo                                                               
1                                  1133                      10282   685.0   
2                                  1416                       9661  9135.0   
3                                  1812                      18225   517.0   
4                                  1210                       8779  5048.0   
5                                  1427                      10549  1236.0   
6                                  1504                       9299   726.0   
7                                  2044                      13272   923.0   
8                                  1054                       9353   596.0   
9                                  1168                      10445  1350.0   
10                                 1422                      13495   754.0   
11                                 1176                       9213  2704.0   
12                                 1297                       9964  2131.0   
13                                 1530                      12543  1457.0   
14                                 1340                       9103   703.0   
15                                 1753                      11967  1428.0   

                                                                            \
               median sqft    mean sqft frac_gas_heating frac_elec_heating   
HousingGroupNo                                                               
1                   2176.0  2386.508029              NaN               NaN   
2                   1690.0  1999.694691              NaN               NaN   
3                   3301.0  4057.108317              NaN               NaN   
4                   1690.0  1629.042789              NaN               NaN   
5                   2176.0  2345.281553              NaN               NaN   
6                   1220.0  1460.060606              NaN               NaN   
7                   2176.0  2549.022752              NaN               NaN   
8                   2176.0  2243.300336              NaN               NaN   
9                   2176.0  3009.053333              NaN               NaN   
10                  2663.0  3614.343501              NaN               NaN   
11                  1220.0  1616.292899              NaN               NaN   
12                  1690.0  2019.074613              NaN               NaN   
13                  1690.0  2281.485930              NaN               NaN   
14                  1220.0  1402.300142              NaN               NaN   
15                  2176.0  2322.710784              NaN               NaN   

                                 HVAC Cooling Type  ...  \
               frac_elec_cooling        Central AC  ...   
HousingGroupNo                                      ...   
1                            NaN   

### Write output to folder

In [28]:
#out_name = "Groups_results" + "_" + baseline_result.split("_")[2]
out.to_csv(os.path.join(result_path, iteration_path, 'buidstock_prototype_results.csv'), index=True)
print(f'file saved to:\n  {os.path.join(result_path, iteration_path)}')

file saved to:
  /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_2007_11
